In [1]:
%load_ext autoreload
%autoreload 2

from model import MLPModel, Config
from utils import define_scheduler_lambda

In [122]:
scheduler_epochs = [2, 2, 8]
scheduler_lambda = define_scheduler_lambda(*scheduler_epochs, min_lambda = 0.03)

config = Config(d_model = 300, 
                weight_decay = 0.5,
                latent_noise = 0.33, 
                epochs=sum(scheduler_epochs) + 10, 
                scheduler_lambda=scheduler_lambda)
model = MLPModel(config).to('mps')

In [123]:
model.fit()

Evaluation | Accuracy: 0.15 %, Loss: 6.6657
Epoch [1/22], Step [100/600], Loss: 5.6981
Epoch [1/22], Step [200/600], Loss: 2.0292
Epoch [1/22], Step [300/600], Loss: 1.2297
Epoch [1/22], Step [400/600], Loss: 1.0258
Epoch [1/22], Step [500/600], Loss: 0.8805
Epoch [1/22], Step [600/600], Loss: 0.6196
learning rate = 0.00017320508075688773
Evaluation | Accuracy: 86.32 %, Loss: 0.6075
Epoch [2/22], Step [100/600], Loss: 0.4897
Epoch [2/22], Step [200/600], Loss: 0.4420
Epoch [2/22], Step [300/600], Loss: 0.4681


KeyboardInterrupt: 